In [1]:
from PINN.PhysicsInformedROPDF import *
# Testing
import matplotlib.pyplot as plt
import torch
import numpy as np
import scipy

# set random seeds
np.random.seed(10)
torch.manual_seed(10);

In [2]:
data_path = "../data/LinearOscillator/OU_Noise_Energy_PINN_Data.mat"
# create PINN
pinn = PhysicsInformedROPDF(indim=2, outdim=1, data_path=data_path, scheduler="ExponentialLR")

In [ ]:
# testing data loss
# take random sample of data points
k = 1000
idx = np.random.choice(len(pinn.pmc), k)
inputs_k = pinn.inputs[idx, :]
pmc_k = pinn.pmc[idx]
pinn.data_loss(inputs_k, pmc_k, gamma=1.0)

In [ ]:
# testing gradient
k = 1000
idx = np.random.choice(len(pinn.pmc), k)
inputs_k = pinn.inputs[idx, :]
p_pred = pinn(inputs_k)
pinn.gradient(p_pred, inputs_k)

In [ ]:
# testing PDE loss
k = 1000
idx = np.random.choice(len(pinn.pmc), k)
inputs_k = pinn.inputs[idx, :]
pinn.domain_loss(inputs_k)

In [ ]:
# testing regularity loss
pinn.regularity_loss()

In [ ]:
# testing training
info = train(
    pinn, pinn.optimizer, pinn.scheduler, batch_size=2**8,
    epochs=50, batch_print=50
)

------------------------------------------------------------------

|                      Epoch 1                                  |

------------------------------------------------------------------

| => | Batch 1 |

| => | Batch 51 |

| => | Batch 101 |

| => | Batch 151 |

| => | Batch 201 |

| => | Batch 251 |

| => | Batch 301 |

| => | Batch 351 |

------------------------------------------------------------------

|        Epoch 1, Batch Average Loss = 227.71432849799623                       |

------------------------------------------------------------------

                     P Loss             = 0.19818430635058268
                     |    domain        = 0.19818430635058268
                     D Loss             = 227.51614419164565
                     R Loss             = 3167312850.3046823
------------------------------------------------------------------

|                      Epoch 2                                  |

----------------------------------------

| => | Batch 51 |

| => | Batch 101 |

| => | Batch 151 |

| => | Batch 201 |

| => | Batch 251 |

| => | Batch 301 |

| => | Batch 351 |

------------------------------------------------------------------

|        Epoch 11, Batch Average Loss = 0.01165226412661973                       |

------------------------------------------------------------------

                     P Loss             = 1.9814879538046506e-06
                     |    domain        = 1.9814879538046506e-06
                     D Loss             = 0.011650282638665924
                     R Loss             = 179013.463201124
------------------------------------------------------------------

|                      Epoch 12                                  |

------------------------------------------------------------------

| => | Batch 1 |

| => | Batch 51 |

| => | Batch 101 |

| => | Batch 151 |

| => | Batch 201 |

| => | Batch 251 |

| => | Batch 301 |

| => | Batch 351 |

---------------------------

| => | Batch 51 |

| => | Batch 101 |

| => | Batch 151 |

| => | Batch 201 |

| => | Batch 251 |

| => | Batch 301 |

| => | Batch 351 |

------------------------------------------------------------------

|        Epoch 21, Batch Average Loss = 9.4414200577064e-06                       |

------------------------------------------------------------------

                     P Loss             = 4.6153284283378643e-07
                     |    domain        = 4.6153284283378643e-07
                     D Loss             = 8.979887214872614e-06
                     R Loss             = 90.33251359680504
------------------------------------------------------------------

|                      Epoch 22                                  |

------------------------------------------------------------------

| => | Batch 1 |

| => | Batch 51 |

| => | Batch 101 |

| => | Batch 151 |

| => | Batch 201 |

| => | Batch 251 |

| => | Batch 301 |

| => | Batch 351 |

-------------------------

In [ ]:
# visualize losses
plt.figure(1, figsize=(16, 8));
plt.plot(np.log(info["pde_loss"]), lw=2, color="red");

plt.figure(2, figsize=(16, 8));
plt.plot(np.log(info["data_loss"]), lw=2, color="blue");

plt.figure(3, figsize=(16, 8));
plt.plot(np.log(info["reg_loss"]), lw=2, color="purple");

plt.figure(4, figsize=(16, 8));
plt.plot(np.log(info["init_loss"]), lw=2, color="purple");

plt.figure(5, figsize=(16, 8));
plt.plot(np.log(info["boundary_loss"]), lw=2, color="purple");

In [ ]:
query = cartesian_data(pinn.tgrid, pinn.xgrid)

D_pred = pinn.D_nn(query).reshape(pinn.nx, pinn.nt).T.detach().numpy()
G_pred = pinn.G_nn(query).reshape(pinn.nx, pinn.nt).T.detach().numpy()
p_pred = pinn(query).reshape(pinn.nx, pinn.nt).T.detach().numpy()

# grids
tgrid = pinn.tgrid.detach().numpy()
xgrid = pinn.xgrid.detach().numpy()

In [ ]:
plt.figure(1);
plt.contourf(xgrid, tgrid, D_pred);
plt.colorbar()
#plt.savefig("./fig/diffusion_nn.png", dpi=100)

In [ ]:
plt.figure(1);
plt.contourf(xgrid, tgrid, G_pred);
plt.colorbar()
#plt.savefig("./fig/drift_nn.png", dpi=100)

In [ ]:
idx = 0
p_exact = pinn.raw_data["pmc"]

plt.plot(p_pred[idx, :], label="pred")
plt.plot(p_exact[idx, :], label="truth")
plt.legend()

In [ ]:
plt.contourf(p_pred)

In [ ]:
plt.contourf(p_exact)

In [ ]:
xgrid = pinn.xgrid.detach().numpy()
dx = xgrid[1]-xgrid[0]
plt.figure(1);
fig, ax = plt.subplots(2, 2);
ax[0, 0].plot(xgrid, p_exact[0, :], label="KDE", lw=1.0, color="red");
ax[0, 0].plot(xgrid, p_pred[0, :], label="PINN", lw=1.0, color="blue");
ax[0, 0].set_title(r"$t = 0$");

ax[0, 1].plot(xgrid, p_exact[125, :], label="KDE", lw=1.0, color="red");
ax[0, 1].plot(xgrid, p_pred[125, :], label="PINN", lw=1.0, color="blue");
ax[0, 1].set_title(r"$t = 2.5$");

ax[1, 0].plot(xgrid, p_exact[250, :], label="KDE", lw=1.0, color="red");
ax[1, 0].plot(xgrid, p_pred[250, :], label="PINN", lw=1.0, color="blue");
ax[1, 0].set_title(r"$t = 5.0$");

ax[1, 1].plot(xgrid, p_exact[-1, :], label="KDE", lw=1.0, color="red");
ax[1, 1].plot(xgrid, p_pred[-1, :], label="PINN", lw=1.0, color="blue");
ax[1, 1].set_title(r"$t = 10.0$");

fig.set_figwidth(8);
fig.tight_layout(pad=0.5);

plt.savefig("./fig/PINN.png", dpi=100);

In [ ]:
plt.figure(2);
# relative error in L^2 from KDE benchmark
l2_rel_err = np.sum(((p_pred-p_exact)**2)*dx, 1) / np.sum((p_exact**2)*dx, 1)
tgrid = pinn.tgrid.detach().numpy()
plt.plot(tgrid, l2_rel_err, color="red", lw=1.5);
plt.grid(True);
plt.title(r"Relative error in $L^2$");
plt.xlabel(r"$t$"); 
plt.ylabel(r"$L^2(\mathbb{R})$ error");
plt.savefig("./fig/PINN_rel_l2_err.png", dpi=100);

## To do

Simulate a constant advection, verify that the PINN implementation works (in terms of identifying the constant advection).